In [1]:
rootF = 'D:/Data/Paper defocus/'

In [2]:
import os as os
import sys
import numpy as np
import csv
import pickle as pickle
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import matplotlib.patches as pcs
from skimage.transform import rescale, resize, downscale_local_mean
import seaborn as sns
from matplotlib.patches import Ellipse

# Import functions
from shared.functions import fit_gaussian, get_ellipse_parameters, plot_scale_bar, load_image_as_on_MEA
from shared.LSC_functions import local_spatial_contrast
from shared.save_load import load_obj, save_obj

In [3]:
exp = 20230105

# Load data

In [4]:
simulation = 'Rem20240325_MouseSA' 

imF = rootF + 'Eye model simulations/convolved images/' + simulation + '/'
path_to_PSFs =  rootF + '/Eye model simulations/mouse PSFs/' + simulation + "/"

##### Simulation parameters

In [5]:
# Load job ids and optical conditions
simulation_parameters = pd.read_csv(path_to_PSFs + 'JMfile.csv', sep=";")
JobIds = simulation_parameters['JobId'].values
# eye SA
eye_SA_all = simulation_parameters['c04tg'].values
eye_SAs = np.unique(eye_SA_all)
# pup diams
pup_diam_all = simulation_parameters['pupDiam_mm'].values
pup_diams = np.unique(pup_diam_all)
# source eccentricity
sources_ecc_all = simulation_parameters['sourceEccen_deg'].values
sources_ecc = np.unique(sources_ecc_all)
# foveal defocus
offset_file = pd.read_csv(path_to_PSFs + 'offset.csv', sep=";")
offset_diopter = offset_file['defoc_diopter'].values
offset_diopter_idx = offset_file['Index'].values

print (len(JobIds), 'jobs in simulation', simulation)
print ('Spherical aberrations:', eye_SAs)
print ('Pup diams:', pup_diams)
print ('Source ecc:', sources_ecc)
print ('Foveal defocus:', offset_diopter)

NSA = len(eye_SAs)
Ndfs = len(offset_diopter)
Npup = len(pup_diams)
NsrcEcc = len(sources_ecc)

mouse_dF_to_um_ratio = -5 #um
mouse_offset = np.linspace(40,-40,9) * mouse_dF_to_um_ratio

# Original images
ImageNbs = [1, 2, 3, 5]
Nimgs = len(ImageNbs)

48 jobs in simulation Rem20240325_MouseSA
Spherical aberrations: [0.   0.15 0.45 0.6 ]
Pup diams: [0.8 1.4 1.5]
Source ecc: [ 0 10 20 30]
Foveal defocus: [ 40.  30.  20.  10.  -0. -10. -20. -30. -40.]


##### Cells with defined STA

In [6]:
exp_path = rootF + '/MEA/exp' + str(exp) + '/'

cells_clustering = load_obj(exp_path + str(exp) + '_selected_cells_clustering')
cells_clustering = np.array(cells_clustering)

print (len(cells_clustering), 'cells were typed')

143 cells were typed


# Local contrast and mean intensity in blurred images

In [7]:
pred_path = rootF + '/contrast analysis/' + simulation + '_local_contrast_2sigma_liu2022_exp' + str(exp) 

if exp == 20211129:
    skip_cells = [45, 80, 188, 192, 401, 408, 422, 525, 533, 558, 561, 614, 682]
elif exp == 20211207:
    skip_cells = [33, 52, 145, 302, 340, 406, 414, 515]
elif exp == 20211208:
    skip_cells = [99,100,276,407,424,452,472,498] # also 407, 452, 472 because too small ellipses
elif exp == 20221213:
    skip_cells = [55, 180, 232, 294, 339, 365, 563, 570, 665, 680]
elif exp == 20221214:
    skip_cells = [40, 190, 332, 957]
elif exp == 20230105:
    skip_cells = [15, 96, 177, 221, 274, 320, 326, 389, 498, 558, 571, 689, 700, 791, 832, 856, 923, 928, 1018]

# Load ellipse points within the 2sigma contour
ell_path = rootF + '/contrast analysis/ellipses_data_with_weights_2sigma_' + str(exp) 

ellipse_data_2s = load_obj(ell_path + '.pkl')

if os.path.isfile(pred_path + '.pkl'):
    print ('Loading predictions')
    local_contrasts = load_obj(pred_path + '.pkl')
    mean_intensities = load_obj(rootF + '/contrast analysis/' + simulation + '_mean_intensities_2sigma_liu2022_exp' + str(exp) + '.pkl')
else:
    print ('Calculating predictions...')

    local_contrasts = np.empty((len(cells_clustering),NSA,Nimgs,Npup,NsrcEcc,Ndfs))
    mean_intensities = np.empty((len(cells_clustering),NSA,Nimgs,Npup,NsrcEcc,Ndfs))

    for index in range(len(cells_clustering)):
        cell_nb = cells_clustering[index]
        print (cell_nb)
        if cell_nb not in skip_cells:
            ### Get ellipse points
            if exp in [20221213, 20221214, 20230105]:
                cell_nb = str(cell_nb)
            ellipsepoints = ellipse_data_2s[cell_nb]['ellipse_points']
            weights = ellipse_data_2s[cell_nb]['weights']
            ### Local contrast for each image
            for iSA in range(NSA):
                for iImg in range(Nimgs):
                    for iPup in [1]: #range(Npup):
                        for iEcc in range(NsrcEcc):
                            for iDf in range(Ndfs):
                                df = str(offset_diopter[iDf])
                                # load blurred image
                                im = load_image_as_on_MEA(imF + 'image_{}eyeSA={}_pupDiam={}_dF='.format(ImageNbs[iImg], eye_SAs[iSA], \
                                                pup_diams[iPup]) + df + '_Source{}.png'.format(sources_ecc[iEcc]))

                                ellipse_weights = weights[ellipsepoints[:,1], ellipsepoints[:,0]]
                                mean_int, lsc = local_spatial_contrast(im, ellipsepoints, ellipse_weights)

                                mean_intensities[index,iSA,iImg,iPup,iEcc,iDf] = mean_int
                                local_contrasts[index,iSA,iImg,iPup,iEcc,iDf] = lsc

Calculating predictions...
5
8
13
15
18
47
48
54
89
96
117
122
134
137
141
148
152
160
169
177
201
215
218
221
230
233
240
248
255
269
271
272
273
274
281
282
284
295
301
308
311
320
326
329
330
345
347
355
356
368
389
394
396
398
404
406
410
415
420
422
424
425
451
454
462
469
482
498
508
510
540
550
553
556
557
558
562
565
569
571
577
578
580
583
585
586
589
590
596
598
600
609
620
642
657
663
689
700
704
719
733
741
744
759
765
777
791
798
817
829
832
856
878
880
900
903
906
921
923
928
929
936
937
943
944
947
951
957
962
964
966
970
975
976
979
982
990
998
1005
1010
1018
1030
1039


In [8]:
### Save predictions
save_path = rootF + '/contrast analysis/' + simulation + '_'
                
save_obj(local_contrasts, save_path + 'local_contrast_2sigma_liu2022_exp' + str(exp))

save_obj(mean_intensities, save_path + 'mean_intensities_2sigma_liu2022_exp' + str(exp))